In [ ]:
import cv2
import pyautogui
import time
import mss
import numpy as np
from matplotlib import pyplot as plt

pyautogui.FAILSAFE = True

---
# Work in progress

In [ ]:
s = mss.mss()
s.monitors

In [ ]:
# image = pyautogui.screenshot()  # Captured image in PIL
# cv_image = np.array(image)  # Cv2 compatible image
# i = cv_image[:, :, ::-1]  # Convert RGB to BGR 

In [ ]:
# Setup
threshold = 0.2  # Acceptable error 
min_th, max_th = (1-threshold, 1+threshold)
img_path = '/home/jakub/Pictures/mahjong.png'
pattern_path = '/home/jakub/Pictures/pattern.png'

In [ ]:
def get_ratios():
    """Helper function to understand a tile shape.
    Unfortunately, resizing game window can change tile shape,
    thus different approach have to be considered."""
    
    img_rgb = cv2.imread(img_path, 0)
    i_w, i_h = img_rgb.shape[::-1]
    template = cv2.imread(pattern_path, 0)
    t_w, t_h = template.shape[::-1]
    rec_ratio = t_w/t_h  # Expected tile ratio
    height_ratio = t_h/i_h  # Expected tile-to-board height ratio
    return rec_ratio, height_ratio, i_w, i_h

rec_ratio, height_ratio, i_w, i_h = get_ratios()

In [ ]:
def grab_screen():
    """mss has to be initialized"""
    img = np.array(sct.grab(monitor))
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    return img

In [ ]:
def loc_tiles(screen):
    """Localize tiles within an image."""
    
    tiles = []
    img_gray = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)
    binary = cv2.bitwise_not(img_gray)
    (contours,_) = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
        if (w/h > (rec_ratio * min_th)
            and w/h < (rec_ratio * max_th)
            and h/i_h > (height_ratio * min_th)
            and h/i_h < (height_ratio * max_th)):
            tiles.append((x, y, h, w))
    print('%d tiles detected.' % len(tiles))
    return tiles

In [ ]:
def try_tile(screen, tile):
    x, y, w, h = tile
    pre_click = screen[y:y+h, x:x+w]
    pyautogui.click(x=(x+w/2), y=(y+h/2))
    post_click = screen[y:y+h, x:x+w]
    difference = cv2.subtract(pre_click, post_click)
    b, g, r = cv2.split(difference)
    if cv2.countNonZero(b) == 0 and cv2.countNonZero(g) == 0 and cv2.countNonZero(r) == 0:
        print("Not clickable")
    else:
        print("Clickable")

In [ ]:
def clickable(tile):
    """Check whether tile is clickable."""
    x, y, w, h = tile

    img = grab_screen()
    pre_click = img[y:y+h, x:x+w]

    pyautogui.click(x=(x+w/2), y=(y+h/2))

    img = grab_screen()
    post_click = img[y:y+h, x:x+w]

    difference = cv2.subtract(pre_click, post_click)
    b, g, r = cv2.split(difference)
    if cv2.countNonZero(b) == 0 and cv2.countNonZero(g) == 0 and cv2.countNonZero(r) == 0:
        return False
    else:
        return tile

In [ ]:
def loc_siblings(tile):
    """Finds matching tiles."""
    x, y, h, w = tile
    screen = grab_screen()
    look_for = screen[y:y+h, x:x+w]
    res = cv2.matchTemplate(screen, look_for, cv2.TM_CCOEFF_NORMED)
    pattern_threshold = 0.8
    loc = np.where(res >= pattern_threshold)
    look = zip(*loc[::-1])
    for pt in look:
        if loc[::-1][0].size == 1:
            print('im lonely')  # Detect unique tiles (need it to detect
                                # gold and black tiles).
            lonely.append(pt)
            pass
        pyautogui.click(x=(pt[0]+w/2), y=(pt[1]+h/2))

In [ ]:
with mss.mss() as sct:
    # Part of the screen to capture
    monitor = {"top": 0, "left": 0, "width": 960, "height": 1080}

    while "Screen capturing":
        last_time = time.time()

        # Get raw pixels from the screen, save it to a np array
        img = grab_screen()

        # Display the picture
        # cv2.imshow("OpenCV/numpyp normal", img)
        tiles = loc_tiles(img)
        clickable_tiles = []
        lonely = []
        for tile in tiles:
            clicked = clickable(tile)
            if clicked is not False:
                clickable_tiles.append(tile)
        
        print('Found {} clickable tiles.'.format(len(clickable_tiles)))
        for tile in clickable_tiles:
            loc_siblings(tile)
        
        print("fps: {}".format(1 / (time.time() - last_time)))

        # Press "q" to quit
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            break

---
# Work in progress

In [ ]:
class Mahjong:
    def __init__(self):
        threshold = 0.2
        self.min_th, self.max_th = (1-threshold, 1+threshold)
        self.img_path = '/home/jakub/Pictures/mahjong.png'
        self.pattern_path = '/home/jakub/Pictures/pattern.png'
        self.rec_ratio, self.height_ratio, self.i_w, self.i_h = self.get_ratios()
        
    def get_ratios(self):
        img_rgb = cv2.imread(self.img_path, 0)
        i_w, i_h = img_rgb.shape[::-1]
        template = cv2.imread(self.pattern_path, 0)
        t_w, t_h = template.shape[::-1]
        rec_ratio = t_w/t_h  # Expected tile ratio
        height_ratio = t_h/i_h  # Expected tile-to-board height ratio
        return rec_ratio, height_ratio, i_w, i_h
    
    def capture_screenshot(self):
        image = pyautogui.screenshot()  # Captured image in PIL
        cv_image = np.array(image)  # Cv2 compatible image
        return cv_image[:, :, ::-1]  # Convert RGB to BGR 
    
    def loc_tiles(self):
        self.tiles = []
        self.lonely = []
        self.screen = self.capture_screenshot().copy()
        img_gray = cv2.cvtColor(self.screen, cv2.COLOR_BGR2GRAY)
        binary = cv2.bitwise_not(img_gray)
        (contours,_) = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        for contour in contours:
            (x,y,w,h) = cv2.boundingRect(contour)
            if (w/h > (self.rec_ratio * self.min_th)
                and w/h < (self.rec_ratio * self.max_th)
                and h/self.i_h > (self.height_ratio * self.min_th)
                and h/self.i_h < (self.height_ratio * self.max_th)):
                self.tiles.append((x, y, h, w))
        print('%d tiles detected.' % len(self.tiles))
    
    def loc_siblings(self, i):
        x, y, h, w = self.tiles[i]
        look_for = self.screen[y:y+h, x:x+w]
        res = cv2.matchTemplate(self.screen, look_for, cv2.TM_CCOEFF_NORMED)
        threshold = 0.8
        loc = np.where(res >= threshold)
        look = zip(*loc[::-1])
        for pt in look:
            if loc[::-1][0].size == 1:
                print('im lonely')
                self.lonely.append(pt)
                break
            pyautogui.click(x=(pt[0]+w/2), y=(pt[1]+h/2))

    def solve(self):
        start = time.time()
        self.loc_tiles()
        for tile in range(len(self.tiles)):
            self.loc_siblings(tile)
        print(self.lonely)
        stop = time.time()
        return stop - start